# Task 1: Simple Detection and Tracking

In [ ]:
# Note: this is colab specific notebook
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.pyplot as plt

!mkdir media/
!wget -nv "https://imgur.com/RfwDZci.png" -O "media/fruit.png"
!gdown "https://drive.google.com/uc?id=1MTXoD5TjltFBgLQHHFa1zVzHZwh3Z5Ti" -O media/track1.mp4

Our task will be detect and localize, i.e find the positions of the cherries in the image and draw a bounding box around them.

The image has already been saved at `media/fruit.png`

![](https://imgur.com/RfwDZci.png)

### Our action plan

- Read the color image
- Convert it to HSV
- Only take the red color part of the image using thresholding
- Find outlines of each cherry
- Draw a box around it

#### Why HSV?

The answer is that unlike RGB, HSV separates luma, or the image intensity, from chroma or the color information. This is very useful in many applications.

In computer vision you often want to separate color components from intensity for various reasons, such as robustness to lighting changes, or removing shadows.

Note: HSV is only one of many color spaces that separate color from intensity (See YCbCr, Lab, etc.). HSV is often used simply because the code for converting between RGB and HSV is widely available and can also be easily implemented

![](https://www.researchgate.net/profile/Ravindran_G/publication/321126312/figure/fig1/AS:561582682722304@1510903153364/llustration-of-the-HSV-Color-Space-B-Color-Feature-Extraction-Color-feature-is-extracted_W640.jpg)

In [ ]:
## Read the image
img = cv2.imread("media/fruit.png",1)
## Display the image
cv2_imshow(img) # or plt.imshow(img) 

In [ ]:
# Convert out RGB image to HSV
hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV )

lower_bound = (40, 39, 0)
higher_bound = (179, 255,255)

# Threshold/mask the img between lower and higher bound
mask = cv2.inRange(hsv_img, lower_bound, higher_bound)

cv2_imshow(mask) # or plt.imshow(img) 

In [ ]:
## Results
# Displays the masked result
res = cv2.bitwise_and(img,img,mask = mask)
cv2_imshow(res) # or plt.imshow(res) 

In [ ]:
## erosion and dilation

# Define a kernel of ones with shape 5X5
kernel = np.ones((5,5))

dilation = cv2.dilate(mask, kernel, iterations=1)
erosion = cv2.erode(dilation, kernel, iterations=2)
dilation = cv2.dilate(erosion, kernel, iterations=2)

plt.figure(figsize=(20,20))
plt.subplot(121),plt.imshow(mask)
plt.subplot(122),plt.imshow(dilation)
plt.show()

mask = dilation.copy()

## Contours 

Contours can be explained simply as a curve joining all the continuous points (along the boundary), having same color or intensity. The contours are a useful tool for shape analysis and object detection and recognition.

For better accuracy, use binary images. So before finding contours, apply threshold or canny edge detection.


In OpenCV, finding contours is like finding white object from black background. So remember, object to be found should be white and background should be black.


In [ ]:
## Finding contours
cnts = cv2.findContours(mask, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2]

img_copy = img.copy()
cv2.drawContours(img_copy, cnts, -1, (0,255,0), 3)
cv2_imshow(img_copy) # or plt.imshow(img_copy) 

In [ ]:
for cnt in cnts:
  # Max are contour and drawing bounding rectangle
  x, y, w, h = cv2.boundingRect(cnt)
  cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)

cv2_imshow(img) # or plt.imshow(img) 

## Tracking in a video

We can use the concepts we just learned above and apply the same to a video.
We are going to try to track the position of the red ball in the below video

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('media/track1.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
out = cv2.VideoWriter(
    "media/track_out" + ".avi", cv2.VideoWriter_fourcc(*"MJPG"), 10.0,
    (1280,720))
cap = cv2.VideoCapture("media/track1.mp4")

while True:
  ## masking
  ret,frame = cap.read()
  if not ret:
    break

  hsv_img = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
  mask = cv2.inRange(hsv_img, (169, 97, 84), (179, 255,255))

  ## Cleaning up the mask
  blur = cv2.GaussianBlur(mask,(5,5),0)
  kernel = np.ones((3,3))
  erosion = cv2.erode(mask,kernel,iterations=1)
  dilation = cv2.dilate(erosion,kernel,iterations=1)
  mask = dilation.copy()

  ## Finding contours
  cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2]

  if cnts:
    # Max area contour and drawing bounding rectangle
    C_max = max(cnts,key = cv2.contourArea)
    x, y, w, h = cv2.boundingRect(C_max)
    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

    ## Draw and find circle radius
    (x,y),radius = cv2.minEnclosingCircle(C_max)
    center = (int(x),int(y))
    radius = int(radius)
    frame = cv2.circle(frame,center,radius,(0,255,0),2)
    #--------------------------------------#

  out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()


In [ ]:
!ffmpeg -hide_banner -loglevel warning -i media/track_out.avi -vf fps="fps=60" media/track_out.mp4 
from IPython.display import HTML
from base64 import b64encode
mp4 = open('media/track_out.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)